# Mixed logit model (MNL)

This notebook was written by Evangelos Paschalidis (evangelos.paschalidis@epfl.ch) for the Decision-aid methodologies in transportation course at EPFL (http://edu.epfl.ch/coursebook/en/decision-aid-methodologies-in-transportation-CIVIL-557). 

Please contact before distributing or reusing the material below.

## Overview

This notebook covers the estimation of a mixed logit model in python with maximum likelihhod estimation:

* Load necessary packages
* Define variables and parameters to estimate
* Model specification
* Model output

Have a go at working through the notebook. To run a code cell, just click on it (to see a green box around it) and then press the **Run** button at the top! 

Some cells have blank lines for you to complete. There is always a comment telling you what to do!

You can also add a new cell by pressing the **+** button at the top of the page.

## Context: Crowdshipping

We have a database of crowdshippers acceptance or rejection of suggested routes. Each crowdshipper is evaluating based on variables such as parcel size, fragility, time of the day, travel time, compensation etc.

## Model specification

## Load packages

Before we estimate the model let's load some packages that we are going to need. When importing a package, it is common to rename it using an abbreviation.

In [12]:
import pandas as pd # for data frame processing
import numpy as np # for some statistical procedures
from scipy.optimize import minimize # optimisation routine for parameter estimation
from scipy.stats import norm # normal distribution density function
import numdifftools as nd # we use this to calculate t-ratios and p-values
from scipy.stats import t # we use this to calculate the CFDs needed for t-ratios and p-values
import csv # we need this to store our parameters as csv

# We've got some rookies in the mix.
from scipy.special import roots_legendre # we use this to generate nodes for numerical integration
from scipy.stats import qmc # we use this to generate nodes for numerical integration

### Let's give a name to our model

In [13]:
model_name = 'MNL_crowdshipping_mixed_solved' # Name we want to give to our model (this is used when saving the output)

### Panel structure
We need to define whether our data is panel (i.e. multiple observations per individual) or not

In [14]:
panel = 0 # switch to 1 if data is panel (any other value if not panel)

### Define if we use mixing

In [15]:
mixing = 1 # switch to 1 if we apply mixing (any other value if no mixing applied)

## Load the data

Now it is time to load the data. We can do that using the piece of code below.

**Important!** Make sure the data is in the same folder with the notebook file

In [16]:
# Command to load the data
data = pd.read_table('data_crowdshipping.txt')

# Number of observations (we need this to caclulate goodness-of-fit indices)
Nobs = data.shape[0]

## Print the data

Let's quickly print the data. Simply type *data* in the field below

(If we open our data set with a different name e.g. *database* then the latter should be used)

* Please make sure to change the name througout the whole script if you change the data frame name.

In [17]:
# Type "data" in this field (without the quotation) and run the cell (Shift + return)


## Print the variable names (columns)

We can also print the variable names only using the piece of code below

* This is useful during model specification to easily access the column names

In [18]:
print(data.columns)

Index(['ID', 'trip_duration', 'time_of_day', 'Size', 'Fragile', 'Compensation',
       'choice'],
      dtype='object')


## Variable definition

We need to define the variables (as numpy arrays) that we will use in our model.

* The arrays can have any name but it is more convenient to use the same name as in the data set.
* The *".reshape(-1, 1)"* is not necessary now but it is required if we include mixing (random effects) in our model.
    * Reshape allows to multiply arrays of different dimensions.

##### Define explanatory (independent) variables

In [19]:
# Example variable_name = np.array(data['variable_name']).reshape(-1, 1)

choice = np.array(data['choice']).reshape(-1, 1)
trip_duration = np.array(data['trip_duration']).reshape(-1, 1)
time_of_day = np.array(data['time_of_day']).reshape(-1, 1)
Size = np.array(data['Size']).reshape(-1, 1)
Fragile = np.array(data['Fragile']).reshape(-1, 1)
Compensation = np.array(data['Compensation']).reshape(-1, 1)

## Add new variables

What if I want to add headway time for all alternatives?

##### Define the ID variable

In [20]:
# Example ID = np.array(data['ID']) # ID does not need to be reshaped

ID = np.array(data['ID']) # ID does not need to be reshaped

##### Delete the data frame
*We may want to delete the data frame to preserve some RAM (suggested for very large data sets)*

In [21]:
# Activate to delete the data frame.
# del[data]

## Model specification

We now need to create a function that includes our model.

* Python functions are defined as: def *function_name*(parameters):
* We end a function as: return *value_to_return*

In the current implementation we specify two different functions as:
* *function 1* calculates the log likelihood per observations
* *function 2* calculates the sum of log likelihood taking as input the result from *function 1*

*We define two different functions to be more flexible in the post estimation processing later in the code*

We use some python (numpy) functions such '*exp*' or '*log*'. To execute these in the current example, we need to also call numpy; hence, we have *np.exp()* and *np.log()*.

### Define parameters and starting values

Ultimately, we want to estimate the value of some parameters that maximise the likelihood of our observations of the dependent variable.

Before starting the estimation process, we need to define some starting values for the parameters to be estimated. The typecal starting value is zero.

In [22]:
betas_start = {"asc": 0, "b_tt_mean": 0, "b_size":0, "b_fragile":0, "b_reward":0,
               "b_morning":0,"b_afternoon":0,"b_evening":0,
                  "sigma_tt":0.1, "sigma":0.1}

### Load old parameter estimates results

Sometimes, we want to use results from old models as starting values.
* To do that, we will load the iteration file from a previous estimation
* Please note that only values of parameters with same names with our current model will be copied

In [23]:
# ### Activate this cell to load old results ###

# # Open old iteration file
# betas_old = pd.read_csv('model_name_iterations.csv') # file with old model iterations (edit accordingly)

# # Keep last row
# betas_old = betas_old.tail(1)

# # Convert to dictionary
# betas_old = betas_old.to_dict(orient='records')[0]

# # Copy values from old to start for keys that are common to both dictionaries
# betas_start = {k: betas_old[k] for k in betas_start.keys() & betas_old.keys()}

# # Delete old estimates
# del[betas_old]

#### Method 2: Code for simulation
The simulation code will generate uniform random numbers (0, 1).

These numbers must be then used in the model function (Function 1) to convert them to out target distribution.

In [24]:
# First let's set the number of draws
# This indicates the number of random draws per random effect
Ndraws = 100

In [25]:
# This piece of code is generating Halton draws using the number of nodes we set just above.
# In this piece of code we define the names we want to use for... 
# ...the random numbers of the random effects.
# We must make changes in the draws = ['draws_1','draws_2'] line of code only

nIndiv = len(set(ID))
draws = ['draws_tt','draws_sigma']
nDim = len(draws)
tasks = (pd.DataFrame(ID).value_counts(sort = False)) # observations per ID

sampler = qmc.Halton(d=nDim, scramble=False)
sample = pd.DataFrame(sampler.random(n=Ndraws*nIndiv+Ndraws)) # +Ndraws
sample = sample[(Ndraws-1):(Ndraws*nIndiv+Ndraws-1)]

cols = len(sample.columns)

for i in range(cols):
    # print(i)
    sample1 = np.array(sample.loc[:,i])
    sample1=pd.DataFrame(np.reshape(sample1,(nIndiv,Ndraws)))
    sample_rep =sample1.loc[sample1.index.repeat(tasks)]
    globals()[draws[i]] = sample_rep

In [26]:
# Convert to normal distribution (or any other preferred)
# Here we use some funciton to convert the uniform draws we obtained
# in the previous block of code to any distirbution
# Example: draws_variable = norm.ppf(draws_variable)
# The norm.ppf() command converts the uniform draws to standard normal

draws_tt = norm.ppf(draws_tt)
draws_sigma = norm.ppf(draws_sigma)

### Function 1: log likelihood (LL)
This function calculates the log likelihood per observation

In [27]:
def LL(betas): # betas is a vector with the parameters we want to estimate
   
    # First let's define the parameters to be estimated.
    # The parameter names are imported directly from 'beta_start' that we defined earlier
    
    for pn in range(len(betas_start.values())):
        globals()[np.array(list(betas_start.keys()))[pn]] = betas[pn]
        
    # Then we need to define the main model specification
    
    # We need to start by defining the utility functions
    # Please make sure that you are using the same names for the parameters as those defined in 'betas_start'
    
    b_tt = b_tt_mean + sigma_tt*draws_tt
    
    U_accept = (asc + b_tt*trip_duration + 
            b_size*Size + b_fragile*Fragile + b_reward*Compensation +
                draws_sigma*sigma +
               b_morning*(time_of_day==1) + b_afternoon*(time_of_day==2) + b_evening*(time_of_day==3))
    U_reject = 0
    
    # TASK ALERT!! How about adding the time of the day in the utility function?? #
        # What is the variable type of time of the day? How many new parameters do we need to specify? #
     
    # And we need to take the exponents of utilities as
    U_accept = np.exp(U_accept)
    U_reject = np.exp(U_reject)
    
    # We need the sum of all utilities to calculate our choice probabilities
    U_sum = U_accept + U_reject
    
    # And then we need to calculate our chouce probabilities
    P_accept = U_accept / U_sum
    P_reject = U_reject / U_sum
    
    # The total probability is then:
    P = P_reject * (choice==0) + P_accept * (choice==1)
    
    ############################################################################################################
    ############################################################################################################
    # - Now this below is relevant if we have panel data and apply mixing (Do not change this piece of code!) -#
    if panel == 1:
    # Do it as panel
        P = pd.DataFrame(P)
        P = pd.concat([pd.Series(ID), pd.DataFrame(P)], axis=1, ignore_index=True)
        P.rename(columns={P.columns[0]: 'ID'},inplace=True)
    
        # We take the product of probabilities per individual per draw and then delete the ID column
        P = P.groupby('ID', as_index=False).prod()
        P = P.drop('ID', axis=1)
   
    if mixing == 1:
        # We take the average per row to get the average probability per individual (if mixing == 1)
        
        if panel == 1:
            P['mean'] = P.mean(axis=1)
            P = np.array(P['mean'])
        
        if panel == 0:
            P = pd.DataFrame(P)
            P = pd.concat([pd.Series(ID), pd.DataFrame(P)], axis=1, ignore_index=True)
            P.rename(columns={P.columns[0]: 'ID'},inplace=True)
    
            # We take the product of probabilities per individual per draw and then delete the ID column
            P = P.groupby('ID', as_index=False).prod()
            P = P.drop('ID', axis=1)
            P['mean'] = P.mean(axis=1)
            P = np.array(P['mean'])
            
    P = np.array(P)
    ### --- This is where the panel data approach ends. --- ###
    ############################################################################################################
    ############################################################################################################
    
    # We then take the log of the density function
    logprob = np.log(P)
    
    return logprob

### Function 2: sum of log likelihood (SLL)
This function simply takes the sum of log likelihood that we calculated with the first function

In [28]:
def SLL(betas):
    return -sum(LL(betas))

## Model estimation

### Warnings

Sometimes, optimisation procedures may 'overdo' it with warnings during estimation.
We can supress these with the piece of code below (not always advisable)

In [29]:
import warnings
warnings.filterwarnings("ignore")

### Estimation

Now it is finally time to run our estimation command.
We use an optimisation algorith called 'BFGS'.

**Do not to edit the lines of code below (except optimisation routine if needed).**

In [30]:
# This will give us the initial loglikelihood value as an output
def callback1(betas):
    print("Current log likelihood:", -SLL(betas))

# This function will allow as to store parameter estimates during iterations
# Initialise list to store parameter values
parameter_values = [np.array(list(betas_start.values()))]
# Then define the function
def callback2(betas):    
    parameter_values.append(betas)
    column_names = list(betas_start.keys())
    with open(f'{model_name}_iterations.csv','w',newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(column_names)
        writer.writerows(parameter_values)

# Now let's combine the two callback functions
def combined_callback(betas):
    callback1(betas)
    callback2(betas)
        
print("Initial log likelihood:", -SLL(np.array(list(betas_start.values()))))

# Choose optimisation routine (preferably BFGS)
optimiser = 'L-BFGS-B' # BFGS or L-BFGS-B or nelder-mead

result = minimize(SLL, np.array(list(betas_start.values())), method=optimiser,callback=combined_callback, 
                  options={'disp':False}) # ,bounds=bounds1
#args = (parameter_values,)
print("Final log likelihood:", -result.fun)

Initial log likelihood: -6930.697502364465
Current log likelihood: -6701.42692627947
Current log likelihood: -6686.511743001411
Current log likelihood: -6682.779291978648
Current log likelihood: -6671.482092242548
Current log likelihood: -6651.494381996537
Current log likelihood: -6595.972458375579
Current log likelihood: -6563.74598816578
Current log likelihood: -6504.871483067333
Current log likelihood: -6363.319537449224
Current log likelihood: -6345.772192635529
Current log likelihood: -6322.096118628811
Current log likelihood: -6317.301391077239
Current log likelihood: -6315.446325205706
Current log likelihood: -6315.037718261939
Current log likelihood: -6314.94043719116
Current log likelihood: -6314.815148441102
Current log likelihood: -6314.490109827133
Current log likelihood: -6313.833354549357
Current log likelihood: -6312.792262448648
Current log likelihood: -6312.336438455155
Current log likelihood: -6310.577434814391
Current log likelihood: -6310.110605139542
Current log li

## Post estimation processing

We evaluate our parameter estimates using t-ratios (or p-Values).

In maximum likelihood estimation, we extract these from the variance-covariance matrix of the parameters.

The variance covariance matrix is not readily available but we need to calculate it.

This is done with the code below.

**DO NOT EDIT THE CHUNK OF CODE BELOW!**

In [31]:
# Vector of parameter estimates
parameters = result['x'] 

# Calculate hessian
print("Calculating Hessian, please wait (this may take a while...)")
Hess = nd.Hessian(SLL)
hessian = Hess(parameters)
inv_hessian = np.linalg.inv(hessian)

# Parameter statistics
dof = Nobs - len(betas_start) - 1
se = np.sqrt(np.diag(inv_hessian)) # Standard errors
tratio = parameters/se # t-ratios
p_value = (1-t.cdf(np.abs(tratio),dof)) * 2 # p-values


# --- Sandwich estimator --- #

# The sandwich estimator provides the "robust" s.e., t-ratios and p-values.
# These should be preferred over the classical parameter statistics.

# We first need the gradients at the solution point
Grad = nd.Gradient(LL)
gradient = Grad(parameters)

# Then we need to calculate the B matrix
B = np.array([])
for r in range(gradient.shape[0]):
    Bm = np.zeros([len(betas_start),len(betas_start)])
    gradient0 = gradient[r,:]
    for i in range(len(gradient0)):
            for j in range(len(gradient0)):
                element = gradient0[i]*gradient0[j]
                Bm[i][j] = element
    if B.size==0:
                    B = Bm
    else:
                    B = B+Bm

# Finally we "sandwich" the B matrix between the inverese hessian matrices
BHHH = (inv_hessian)@(B)@(inv_hessian)

print("... Done!!")

# Now it is time to calculate some "robust" parameter statistics
rob_se = np.sqrt(np.diag(BHHH)) # robust standard error
rob_tratio = parameters/rob_se # robust t-ratio
rob_p_value = (1-t.cdf(np.abs(rob_tratio),dof)) * 2 # robust p-value

Calculating Hessian, please wait (this may take a while...)
... Done!!


## Results
Finally, we got our results. Let's print them!

The outputs that we receive are:
* Estimates: These are the values of our parameters. We must check if the sign is consistent with our expectations
* s.e.: Standard errors of the parameters
* tratio: t-ratio of the parameters (significant if absolute value > 1.96)
* p_value: p-value of the parameters (significant if < 0.05)

The parameter statistics also have their **robust** versions. These should be preferred as they are less susceptible to weakly specified models.

In [32]:
arrays = np.column_stack((np.array(list(betas_start.keys())),parameters,se,tratio,p_value,rob_se,rob_tratio,rob_p_value))
results = pd.DataFrame(arrays, columns = ['Parameter','Estimate','s.e.','t-ratio0','p-value',
                                          'Rob s.e.','Rob t-ratio0','Rob p-value'])

results[['Estimate','s.e.','t-ratio0','p-value','Rob s.e.','Rob t-ratio0','Rob p-value']] = (
results[['Estimate','s.e.','t-ratio0','p-value','Rob s.e.','Rob t-ratio0','Rob p-value']].apply(pd.to_numeric, errors='coerce'))
numeric_cols = results.select_dtypes(include='number').columns
results[numeric_cols] = results[numeric_cols].round(3)
results # print results

,Parameter,Estimate,s.e.,t-ratio0,p-value,Rob s.e.,Rob t-ratio0,Rob p-value
0,asc,-1.371,0.132,-10.399,0.000,0.140,-9.825,0.000
1,b_tt_mean,-0.004,0.002,-1.749,0.080,0.002,-1.743,0.081
2,b_size,1.154,0.155,7.447,0.000,0.158,7.293,0.000
3,b_fragile,0.995,0.060,16.622,0.000,0.062,15.965,0.000
4,b_reward,0.028,0.005,5.454,0.000,0.005,5.337,0.000
5,b_morning,0.669,0.074,9.094,0.000,0.075,8.968,0.000
6,b_afternoon,1.381,0.088,15.744,0.000,0.093,14.891,0.000
7,b_evening,1.087,0.080,13.600,0.000,0.084,13.016,0.000
8,sigma_tt,0.001,0.017,0.042,0.967,0.009,0.082,0.935
9,sigma,0.062,1.330,0.047,0.963,1.471,0.042,0.966


## Goodness-of-fit indices

Let's calculate some goodness-of-fit indices now (do not edit)

In [33]:
# First let's calculate the GoF indices

rho_squared = 1 - ((-result.fun)/(-SLL(np.zeros(len(betas_start)))))
adj_rho_squared = 1 - (((-result.fun)-len(betas_start))/(-SLL(np.zeros(len(betas_start)))))

AIC = 2*len(betas_start) - 2*(-result.fun)
BIC = len(betas_start)*np.log(Nobs) - 2*(-result.fun)

LL0t = "Log likelihood at zeros:" + str(-SLL(np.zeros(len(betas_start))))
LLinit = "Initial log likelihood:" + str(-SLL(np.array(list(betas_start.values()))))
LLfin = "Final log likelihood:" + str(-result.fun)

rs1 = "rho squared="+str(rho_squared)
rs2 = "adjusted rho squared="+str(adj_rho_squared)
ac = "AIC="+str(AIC)
bc = "BIC="+str(BIC)

# Then let's print the GoF indices

print(LL0t)
print(LLinit)
print(LLfin)

print(rs1)
print(rs2)
print(ac)
print(bc)

Log likelihood at zeros:-6931.471805600547
Initial log likelihood:-6930.697502364465
Final log likelihood:-6177.086884362558
rho squared=0.10883473847912861
adjusted rho squared=0.10739204343823994
AIC=12374.173768725115
BIC=12446.277172444878


## Save output

We can save our output using the code below (do not edit):

In [34]:
with open(f"{model_name}_results.txt", 'w') as f:
    f.write(f'{LL0t}\n')
    f.write(f'{LLinit}\n')
    f.write(f'{LLfin}\n')
    f.write(f'{rs1}\n')
    f.write(f'{rs2}\n')
    f.write(f'{ac}\n')
    f.write(f'{bc}\n')
    results.to_csv(f, index=False, sep='\t')
results.to_csv(f'{model_name}_results.csv', index=False)